In [25]:
import os
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.io import WriteToText
import string

table_spec = '`bigquery-public-data.hacker_news.full`'

options=PipelineOptions()

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jupyter-albert.yumol@gmail-e7c4e/eskwelabshack-5b0b7c34019a.json'
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

options.view_as(SetupOptions).save_main_session = True
options.view_as(GoogleCloudOptions).project = "eskwelabshack"
# options.view_as(StandardOptions).runner = 'DataflowRunner'
options.view_as(GoogleCloudOptions).temp_location='gs://eskwelabshack/temp'
options.view_as(GoogleCloudOptions).staging_location='gs://eskwelabshack/staging'


query = 'SELECT title FROM ' + table_spec + ' where type = ' + '\'job\' and title is not null order by score limit 100;'
print(query)
with beam.Pipeline(options=options) as p:
   result = (
   p
   | "Read Table" >> beam.io.Read(beam.io.BigQuerySource(
       query = query, use_standard_sql = True))
   | beam.Map(lambda x: x['title'])
   | beam.Map(lambda x:''.join([i for i in x if i not in string.punctuation]))
   | beam.FlatMap(lambda x: x.lower().split(' '))
   | "Map for Sum" >> beam.Map(lambda x: (x, 1))
   | beam.CombinePerKey(sum)
   | WriteToText('gs://eskwelabshack/output/word-count',num_shards = 1)
   )

/home/jupyter-albert.yumol@gmail-e7c4e/eskwelabshack-5b0b7c34019a.json
SELECT title FROM `bigquery-public-data.hacker_news.full` where type = 'job' and title is not null order by score limit 100;


In [2]:
import wordcloud

ModuleNotFoundError: No module named 'wordcloud'

In [6]:
import os
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
options=PipelineOptions()
with beam.Pipeline(options=options) as p:
    result = (
    p
    | beam.Create([3,2,1,5,4])
    | "test" >> beam.ParDo(lambda x: [x*x])
    | beam.ParDo(lambda x: print(x))
    )

9
4
1
25
16
